In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.document_loaders import PyPDFLoader
def load_pdf_files(files):
    # Assuming the provided 'files' list contains URLs
    pdf_url = files[0]

    # Read PDF content from the URL
    pdf_loader = PyPDFLoader(pdf_url)
    pages = pdf_loader.load_and_split()

    # If there are more files in the list, process them
    for i in range(1, len(files)):
        pdf_url = files[i]
        pdf_loader = PyPDFLoader(pdf_url)
        pages += pdf_loader.load_and_split()

    return pages

In [4]:
files=["IPC_186045.pdf"]
pages=load_pdf_files(files)
print("pages in PDF files: ",len(pages))

pages in PDF files:  187


In [5]:
from langchain.prompts import PromptTemplate
template = """

{context}


Question: {question} with section number. add all details.
Answer:"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [7]:
from langchain.vectorstores import FAISS
from langchain.embeddings import  HuggingFaceInstructEmbeddings,HuggingFaceHubEmbeddings

# HFIembeddings = HuggingFaceInstructEmbeddings(model_name="WhereIsAI/UAE-Large-V1")
HFIembeddings = HuggingFaceHubEmbeddings(repo_id="sentence-transformers/all-mpnet-base-v2")
# HFIembeddings = HuggingFaceInstructEmbeddings(model_name="thenlper/gte-small",cache_folder="./Models/")
# HFIembeddings = HuggingFaceInstructEmbeddings(model_name="thenlper/gte-small",cache_folder="./Models/")
vectorstore = FAISS.from_documents(pages,embedding=HFIembeddings)
print("vectore store created")

You're using a different task than the one specified in the repository. Be sure to know what you're doing :)


vectore store created


In [23]:

from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
chain = RetrievalQA.from_chain_type(
    llm = HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", model_kwargs={"temperature":0.8, "max_length":4096,"max_new_tokens":4096}),
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 10}),    
    # retriever=vectorstore.as_retriever(),    
    return_source_documents=True,
    # kwargs={"max_length":4096,"min_length":1024},
    chain_type_kwargs={"prompt": prompt,"verbose": True},
)

In [24]:
# q="what is the punishment for robbery ?"
# q="who is judge ?"
# q="one person has commited defamation, what should be punishment for it?"
# q="rioting has happened in neighbouring society. who will be responsible and what is the punishment for it"
# q="what is punishment for robbery and also murder ?"
q="person is selling adulterated drugs which is harmful to the health of people. person has also sold drug to the children below age of 15 what will be punishment for it?"
# q+=" with section number. add all details."
# q+=" including section number and all related details."
response = chain({"query":q, "early_stopping":True,"min_length":5000,"max_tokens":5000})

from write_in_file import generate_docx_with_bullets
from data_extracter import extract_data_from_response
result,src_data,src_pg_nms=extract_data_from_response(response)
generate_docx_with_bullets(heading=q,main_paragraph=result,srcs=src_pg_nms,output_folder="./tmp/")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


imprisonment of either description for a term which may extend to six months, or with fine which may 
extend to one  thousand rupees, or with both.  
275. Sale of adulterated drugs .—Whoever, knowing any drug or medical preparation to have been 
adulterated in such a manner as to lessen its efficacy, to change its operation, or to render it noxious, sells 
                                                           
1. The word “and” omitted by Act 42 of 1953, s. 4 and the Third Sch.  
2. Subs. by the A. O. 1937, for “by the G. of I., or by any Government”.  
3. The words “Central or any Provincial” omitted by the A. O. 1950.  
4. The words “or the Crown Representative” omitted by the A. O. 1948.

66 
 the same, or offers or exposes it for sale, or issues it from any dispensary for m edicinal purposes as 
unadulterated, or causes it to be used for medicinal purposes by any person 

In [25]:
# def write_in_notepad(response):
#     from write_in_file import open_in_notepad
#     open_in_notepad(response["result"])


